In [2]:
cd c:\Users\rio_m\OneDrive - 国立大学法人東海国立大学機構\100knock\hlab2023-nlp100

c:\Users\rio_m\OneDrive - 国立大学法人東海国立大学機構\100knock\hlab2023-nlp100


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# データの読込
df = pd.read_csv('./newsCorpora.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# データの抽出
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]

# データの分割
train, valid_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=123, stratify=df['CATEGORY'])
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, random_state=123, stratify=valid_test['CATEGORY'])

# 事例数の確認
print('【学習データ】')
print(train['CATEGORY'].value_counts())
print('【検証データ】')
print(valid['CATEGORY'].value_counts())
print('【評価データ】')
print(test['CATEGORY'].value_counts())

【学習データ】
CATEGORY
b    4502
e    4223
t    1219
m     728
Name: count, dtype: int64
【検証データ】
CATEGORY
b    562
e    528
t    153
m     91
Name: count, dtype: int64
【評価データ】
CATEGORY
b    563
e    528
t    152
m     91
Name: count, dtype: int64


In [4]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [13]:
%pip install torch


                                              0.0/172.3 MB ? eta -:--:--
                                              0.3/172.3 MB 6.3 MB/s eta 0:00:28
                                              0.4/172.3 MB 5.3 MB/s eta 0:00:33
                                              0.6/172.3 MB 5.2 MB/s eta 0:00:34
                                              0.6/172.3 MB 3.7 MB/s eta 0:00:47
                                              0.7/172.3 MB 3.2 MB/s eta 0:00:54
                                              0.8/172.3 MB 3.4 MB/s eta 0:00:51
                                              1.0/172.3 MB 3.5 MB/s eta 0:00:49
                                              1.1/172.3 MB 3.4 MB/s eta 0:00:50
                                              1.3/172.3 MB 3.5 MB/s eta 0:00:49
                                              1.5/172.3 MB 3.6 MB/s eta 0:00:48
                                              1.6/172.3 MB 3.5 MB/s eta 0:00:50
                                              

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
import string
import torch

def transform_w2v(text):
  table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  words = text.translate(table).split()
  vec = [model[word] for word in words if word in model]
  return torch.tensor(sum(vec) / len(vec))  # 平均ベクトルをTensor型に変換して出力

In [6]:

X_valid = torch.stack([transform_w2v(text) for text in valid['TITLE']])
X_test = torch.stack([transform_w2v(text) for text in test['TITLE']])
X_train = torch.stack([transform_w2v(text) for text in train['TITLE']])

print(X_train.size())
print(X_train)

torch.Size([10672, 300])
tensor([[-0.0170,  0.1318, -0.0728,  ...,  0.0395,  0.0223,  0.0184],
        [-0.1119, -0.0523, -0.1002,  ...,  0.0319, -0.0237, -0.0425],
        [-0.0636, -0.0228, -0.0005,  ..., -0.0280,  0.1057,  0.0396],
        ...,
        [ 0.0301, -0.0355, -0.0082,  ..., -0.0045,  0.0606, -0.0281],
        [ 0.0002,  0.0442, -0.0424,  ..., -0.0507,  0.0283,  0.0365],
        [ 0.0242,  0.0418,  0.1211,  ..., -0.0564,  0.0144,  0.0277]])


In [9]:
category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
y_train = torch.tensor(train['CATEGORY'].map(lambda x: category_dict[x]).values)
y_valid = torch.tensor(valid['CATEGORY'].map(lambda x: category_dict[x]).values)
y_test = torch.tensor(test['CATEGORY'].map(lambda x: category_dict[x]).values)

print(y_train.size())
print(y_train)

torch.Size([10672])
tensor([0, 0, 0,  ..., 0, 2, 2])


In [10]:
torch.save(X_train, 'X_train.pt')
torch.save(X_valid, 'X_valid.pt')
torch.save(X_test, 'X_test.pt')
torch.save(y_train, 'y_train.pt')
torch.save(y_valid, 'y_valid.pt')
torch.save(y_test, 'y_test.pt')

In [11]:
from torch import nn

class SLPNet(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    self.fc = nn.Linear(input_size, output_size, bias=False)
    nn.init.normal_(self.fc.weight, 0.0, 1.0)  # 正規乱数で重みを初期化

  def forward(self, x):
    x = self.fc(x)
    return x

In [12]:
model = SLPNet(300, 4)  # 単層ニューラルネットワークの初期化
y_hat_1 = torch.softmax(model(X_train[:1]), dim=-1)
print(y_hat_1)

tensor([[0.0836, 0.0105, 0.6869, 0.2190]], grad_fn=<SoftmaxBackward0>)


In [13]:
Y_hat = torch.softmax(model.forward(X_train[:4]), dim=-1)
print(Y_hat)

tensor([[0.0836, 0.0105, 0.6869, 0.2190],
        [0.4820, 0.0553, 0.4568, 0.0058],
        [0.0902, 0.1264, 0.6596, 0.1239],
        [0.1293, 0.0603, 0.4328, 0.3776]], grad_fn=<SoftmaxBackward0>)


In [14]:
criterion = nn.CrossEntropyLoss()


In [15]:
l_1 = criterion(model(X_train[:1]), y_train[:1])  # 入力ベクトルはsoftmax前の値
model.zero_grad()  # 勾配をゼロで初期化
l_1.backward()  # 勾配を計算
print(f'損失: {l_1:.4f}')
print(f'勾配:\n{model.fc.weight.grad}')

損失: 2.4821
勾配:
tensor([[ 0.0155, -0.1207,  0.0667,  ..., -0.0362, -0.0204, -0.0168],
        [-0.0002,  0.0014, -0.0008,  ...,  0.0004,  0.0002,  0.0002],
        [-0.0117,  0.0905, -0.0500,  ...,  0.0271,  0.0153,  0.0126],
        [-0.0037,  0.0288, -0.0159,  ...,  0.0086,  0.0049,  0.0040]])


In [16]:
l = criterion(model(X_train[:4]), y_train[:4])
model.zero_grad()
l.backward()
print(f'損失: {l:.4f}')
print(f'勾配:\n{model.fc.weight.grad}')

損失: 1.9159
勾配:
tensor([[ 0.0389, -0.0506,  0.0311,  ..., -0.0074, -0.0239, -0.0087],
        [-0.0040,  0.0011, -0.0017,  ..., -0.0003,  0.0029,  0.0008],
        [-0.0292,  0.0290, -0.0247,  ...,  0.0061,  0.0174,  0.0053],
        [-0.0057,  0.0205, -0.0047,  ...,  0.0016,  0.0035,  0.0026]])


In [17]:
from torch.utils.data import Dataset

class NewsDataset(Dataset):
  def __init__(self, X, y):  # datasetの構成要素を指定
    self.X = X
    self.y = y

  def __len__(self):  # len(dataset)で返す値を指定
    return len(self.y)

  def __getitem__(self, idx):  # dataset[idx]で返す値を指定
    return [self.X[idx], self.y[idx]]

In [19]:
from torch.utils.data import DataLoader

# Datasetの作成
dataset_train = NewsDataset(X_train, y_train)
dataset_valid = NewsDataset(X_valid, y_valid)
dataset_test = NewsDataset(X_test, y_test)

# Dataloaderの作成
dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=len(dataset_test), shuffle=False)

In [20]:
# モデルの定義
model = SLPNet(300, 4)

# 損失関数の定義
criterion = nn.CrossEntropyLoss()

# オプティマイザの定義
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

# 学習
num_epochs = 10
for epoch in range(num_epochs):
  # 訓練モードに設定
  model.train()
  loss_train = 0.0
  for i, (inputs, labels) in enumerate(dataloader_train):
    # 勾配をゼロで初期化
    optimizer.zero_grad()

    # 順伝播 + 誤差逆伝播 + 重み更新
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # 損失を記録
    loss_train += loss.item()
 
  # バッチ単位の平均損失計算
  loss_train = loss_train / i

  # 検証データの損失計算
  model.eval() 
  with torch.no_grad():
    inputs, labels = next(iter(dataloader_valid))
    outputs = model(inputs)
    loss_valid = criterion(outputs, labels)

  # ログを出力
  print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, loss_valid: {loss_valid:.4f}')  

epoch: 1, loss_train: 0.4802, loss_valid: 0.3618
epoch: 2, loss_train: 0.3205, loss_valid: 0.3221
epoch: 3, loss_train: 0.2903, loss_valid: 0.3037
epoch: 4, loss_train: 0.2746, loss_valid: 0.2913
epoch: 5, loss_train: 0.2640, loss_valid: 0.2874
epoch: 6, loss_train: 0.2568, loss_valid: 0.2792
epoch: 7, loss_train: 0.2501, loss_valid: 0.2764
epoch: 8, loss_train: 0.2461, loss_valid: 0.2735
epoch: 9, loss_train: 0.2431, loss_valid: 0.2754
epoch: 10, loss_train: 0.2397, loss_valid: 0.2721


In [21]:
def calculate_accuracy(model, loader):
  model.eval()
  total = 0
  correct = 0
  with torch.no_grad():
    for inputs, labels in loader:
      outputs = model(inputs)
      pred = torch.argmax(outputs, dim=-1)
      total += len(inputs)
      correct += (pred == labels).sum().item()
      
  return correct / total

In [22]:
acc_train = calculate_accuracy(model, dataloader_train)
acc_test = calculate_accuracy(model, dataloader_test)
print(f'正解率（学習データ）：{acc_train:.3f}')
print(f'正解率（評価データ）：{acc_test:.3f}')

正解率（学習データ）：0.920
正解率（評価データ）：0.906


In [23]:
!git add 6.ipynb

fatal: pathspec '6.ipynb' did not match any files


In [19]:
!git commit -m “commit6”


On branch suzuki
Your branch and 'origin/suzuki' have diverged,
and have 4 and 7 different commits each, respectively.
  (use "git pull" to merge the remote branch into yours)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
  (commit or discard the untracked or modified content in submodules)
	modified:   4.ipynb
	modified:   hlab2023-nlp100 (untracked content)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	GoogleNews-vectors-negative300.bin.gz
	combined.csv
	questions-words-write.txt
	questions-words.txt

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git push orizin suzuki